In [1]:
#from transformers.utils.fx import symbolic_trace
from transformers import AutoConfig, AutoFeatureExtractor, AutoTokenizer, AutoProcessor, AutoModelForSpeechSeq2Seq
from modeling_whisper_traceable import WhisperForConditionalGeneration
from datasets import load_dataset
import torch
from torch.fx import symbolic_trace
import torch.fx as fx


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda112.so
CUDA SETUP: CUDA runtime path found: /home/ujan/anaconda3/envs/asr/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda112.so...


In [1]:
#from transformers.utils.fx import symbolic_trace
from transformers import AutoConfig, AutoFeatureExtractor, AutoTokenizer, AutoProcessor, AutoModelForSpeechSeq2Seq
from modeling_whisper_traceable import WhisperForConditionalGeneration
from datasets import load_dataset

model_name_or_path = 'openai/whisper-small'
data_dir = 'mozilla-foundation/common_voice_11_0'

In [3]:
# load dataset
print('loading dataset from {}'.format(data_dir))

raw_datasets = load_dataset(data_dir, "zh-CN", split="test", streaming=True)
text_column_name = 'sentence'


# model, tokenizer, feature extractor, processor

model_config = AutoConfig.from_pretrained(
    model_name_or_path,
    #cache_dir=args.cache_dir,
    #revision=args.model_revision,
    #use_auth_token=True if args.use_auth_token else None,
)

model_config.update({"forced_decoder_ids": [], "suppress_tokens": []})


feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_name_or_path,
    #cache_dir=args.cache_dir,
    #revision=args.model_revision,
    #use_auth_token=True if args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    #cache_dir=args.cache_dir,
    #use_fast=model_args.use_fast_tokenizer,
    #revision=model_args.model_revision,
    #use_auth_token=True if model_args.use_auth_token else None,
)

tokenizer.set_prefix_tokens(language='chinese', task='transcribe')


model = WhisperForConditionalGeneration.from_pretrained(
    model_name_or_path,
    config=model_config,
    #cache_dir=args.cache_dir,
    #revision=args.model_revision,
    #use_auth_token=True if args.use_auth_token else None,
)

if model.config.decoder_start_token_id is None:
    raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")
    
processor = AutoProcessor.from_pretrained(model_name_or_path)
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language='chinese', task='transcribe')


dataset = raw_datasets

model.eval()


sample = next(iter(dataset))


inputs = processor(
    sample['audio']["array"],
    sampling_rate=feature_extractor.sampling_rate,
    return_attention_mask=True,
    return_tensors="pt")
                
input_features = inputs.input_features
attention_mask = inputs.attention_mask
decoder_input_ids = torch.tensor([model.config.decoder_start_token_id]).reshape(1, -1)

loading dataset from mozilla-foundation/common_voice_11_0


/home/ujan/speech-processing/experiments/notebooks/modeling_whisper_traceable.py:1467: UserWarning: disabled check : 
                attn_weights.size() == (bsz * self.num_heads, tgt_len, src_len) 
            in WhisperAttention.forward
  warnings.warn(
/home/ujan/speech-processing/experiments/notebooks/modeling_whisper_traceable.py:1472: UserWarning: disabled check :
                attn_output.size() == (bsz * self.num_heads, tgt_len, self.head_dim)
            in WhisperAttention.forward
  warnings.warn(
/home/ujan/speech-processing/experiments/notebooks/modeling_whisper_traceable.py:1477: UserWarning: if passing in a tuple for encoder_outputs, wrap it in a BaseModelOutput when return_dict=True
                before passing through model. As :
                    encoder_outputs = BaseModelOutput(
                    last_hidden_state=encoder_outputs[0],
                    hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                    attentions=enco

In [4]:
encoder = model.get_encoder()

In [5]:
encoder_concrete_args={
                'output_hidden_states': False,
                'output_attentions': False,
                'head_mask': None,
                'return_dict': True,
              }

decoder_concrete_args={
                'output_hidden_states': False,
                'output_attentions': False,
                'return_dict': True,
                'use_cache': True,
                'decoder_inputs_embeds': None,
              }

In [6]:
symbolic_traced : torch.fx.GraphModule = symbolic_trace(encoder, concrete_args=encoder_concrete_args)

In [7]:
print(symbolic_traced.code)




def forward(self, input_features, attention_mask = None, head_mask_1 = None, output_attentions_1 = None, output_hidden_states_1 = None, return_dict_1 = None):
    _assert_is_none = torch.fx._symbolic_trace._assert_is_none(head_mask_1, 'head_mask has been specialized to have value None but got another value');  head_mask_1 = None
    eq = output_attentions_1 == False;  output_attentions_1 = None
    _assert = torch._assert(eq, 'output_attentions has been specialized to have value False but got another value');  eq = None
    eq_1 = output_hidden_states_1 == False;  output_hidden_states_1 = None
    _assert_1 = torch._assert(eq_1, 'output_hidden_states has been specialized to have value False but got another value');  eq_1 = None
    eq_2 = return_dict_1 == True;  return_dict_1 = None
    _assert_2 = torch._assert(eq_2, 'return_dict has been specialized to have value True but got another value');  eq_2 = None
    conv1 = self.conv1(input_features);  input_features = None
    gelu = to

In [8]:
symbolic_traced : torch.fx.GraphModule = symbolic_trace(model, concrete_args=decoder_concrete_args)

In [9]:
print(symbolic_traced.code)


torch.fx._symbolic_trace.wrap("modeling_whisper_traceable__prepare_decoder_attention_mask")

def forward(self, input_features : typing_Union[torch.FloatTensor,NoneType] = None, attention_mask : typing_Union[torch.LongTensor,NoneType] = None, decoder_input_ids : typing_Union[torch.LongTensor,NoneType] = None, decoder_attention_mask : typing_Union[torch.LongTensor,NoneType] = None, head_mask : typing_Union[torch.Tensor,NoneType] = None, decoder_head_mask : typing_Union[torch.Tensor,NoneType] = None, cross_attn_head_mask : typing_Union[torch.Tensor,NoneType] = None, encoder_outputs : typing_Union[typing_Tuple[typing_Tuple[torch.FloatTensor]],NoneType] = None, past_key_values : typing_Union[typing_Tuple[typing_Tuple[torch.FloatTensor]],NoneType] = None, decoder_inputs_embeds_1 = None, labels : typing_Union[torch.LongTensor,NoneType] = None, use_cache_1 = None, output_attentions_1 = None, output_hidden_states_1 = None, return_dict_1 = None) -> typing_Union[typing_Tuple[torch.Tensor],transf

In [6]:
from torch.ao.quantization import get_default_qconfig
from quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping

In [7]:
qconfig = get_default_qconfig("x86")
qconfig_mapping = QConfigMapping().set_global(qconfig)

In [8]:
example_inputs = (input_features, attention_mask, decoder_input_ids)

In [9]:
prepare_fx(model, qconfig_mapping, example_inputs, concrete_args=decoder_concrete_args)

/Users/ujan/opt/anaconda3/envs/asr/lib/python3.10/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


GraphModule(
  (activation_post_process_0): HistogramObserver(min_val=inf, max_val=-inf)
  (activation_post_process_1): HistogramObserver(min_val=inf, max_val=-inf)
  (model): Module(
    (decoder): Module(
      (embed_tokens): Embedding(51865, 768, padding_idx=50257)
      (embed_positions): Module()
      (layers): Module(
        (0): Module(
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (self_attn): Module(
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (encoder_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (encoder_attn): Module(
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_

In [2]:
import torch

In [3]:
mask = torch.rand(8,8)
mask

tensor([[0.3670, 0.9916, 0.8972, 0.7970, 0.3965, 0.5842, 0.2741, 0.5255],
        [0.6296, 0.4956, 0.1430, 0.8266, 0.5940, 0.7007, 0.2518, 0.1667],
        [0.7255, 0.0080, 0.9489, 0.1894, 0.5512, 0.5293, 0.2753, 0.0508],
        [0.0584, 0.3933, 0.1139, 0.7904, 0.3747, 0.3514, 0.7094, 0.1210],
        [0.2017, 0.0492, 0.1117, 0.2658, 0.8969, 0.0542, 0.2911, 0.9022],
        [0.3773, 0.9827, 0.1489, 0.0572, 0.9670, 0.2209, 0.4054, 0.3755],
        [0.8901, 0.5765, 0.0042, 0.6808, 0.5876, 0.0787, 0.9588, 0.2276],
        [0.7936, 0.0282, 0.0572, 0.2471, 0.8485, 0.9265, 0.1902, 0.8694]])

In [10]:
(8**2) / (2**2)

16.0

In [12]:
rows = 8
cols = 8
block_size = 2

In [14]:
for b in range(rows//block_size):
    print(b*block_size)

0
2
4
6


In [29]:
row_ids = [b*block_size for b in range(rows//block_size)]

In [30]:
col_ids = [b*block_size for b in range(cols//block_size)]

In [4]:
import itertools

In [43]:
block_starts = list(itertools.product(row_ids, col_ids))

In [36]:
starts[0][0]

0

In [46]:
for starts in block_starts:
    block = mask[starts[0]:starts[0]+block_size, starts[1]:starts[1]+block_size]
    print(block)
    print(block.numel())
    break

tensor([[0.5500, 0.6808],
        [0.8853, 0.0787]])
4


In [45]:
mask.numel()

64

In [47]:
mask

tensor([[0.5500, 0.6808, 0.0000, 0.0000, 0.3190, 0.4696, 0.1576, 0.5488],
        [0.8853, 0.0787, 0.0000, 0.0000, 0.0588, 0.3720, 0.8529, 0.5314],
        [0.5348, 0.5938, 0.3496, 0.1799, 0.9819, 0.6040, 0.5582, 0.3997],
        [0.4339, 0.7161, 0.8938, 0.2811, 0.3184, 0.5575, 0.6046, 0.9316],
        [0.0519, 0.7054, 0.1441, 0.0785, 0.2620, 0.7403, 0.8035, 0.4934],
        [0.6501, 0.0503, 0.0231, 0.4434, 0.7548, 0.2351, 0.0118, 0.4432],
        [0.6796, 0.2638, 0.6830, 0.9232, 0.9234, 0.0189, 0.7614, 0.2667],
        [0.7023, 0.5816, 0.4335, 0.0157, 0.0692, 0.3074, 0.8151, 0.9757]])

In [50]:
x = torch.rand((4,4), requires_grad=True)
x

tensor([[0.0046, 0.6940, 0.1306, 0.8281],
        [0.0148, 0.0548, 0.3951, 0.5808],
        [0.3264, 0.6692, 0.2732, 0.4666],
        [0.0983, 0.0590, 0.1664, 0.3158]], requires_grad=True)

In [52]:
y = torch.var(x)

In [54]:
y

tensor(0.0705, grad_fn=<VarBackward0>)

In [53]:
y.backward(retain_graph=True)
x.grad

tensor([[-0.0417,  0.0502, -0.0249,  0.0681],
        [-0.0403, -0.0350,  0.0104,  0.0351],
        [ 0.0012,  0.0469, -0.0059,  0.0199],
        [-0.0292, -0.0344, -0.0201, -0.0002]])

In [6]:
block_size = 2
out_features = 8
in_features = 8

In [7]:
row_ids = [b*block_size for b in range(out_features//block_size)]
# column indices where a block starts
col_ids = [b*block_size for b in range(in_features//block_size)]
    # cartesian product
# each element is a tuple containing the top left position of each block
block_starts = list(itertools.product(row_ids, col_ids))

In [8]:
block_starts

[(0, 0),
 (0, 2),
 (0, 4),
 (0, 6),
 (2, 0),
 (2, 2),
 (2, 4),
 (2, 6),
 (4, 0),
 (4, 2),
 (4, 4),
 (4, 6),
 (6, 0),
 (6, 2),
 (6, 4),
 (6, 6)]

In [11]:
for starts in block_starts:
    # get a block
    block = mask[starts[0]:starts[0]+block_size, starts[1]:starts[1]+block_size]
    mask[starts[0]:starts[0]+block_size, starts[1]:starts[1]+block_size] = 0
    break

In [12]:
mask

tensor([[0.0000, 0.0000, 0.8972, 0.7970, 0.3965, 0.5842, 0.2741, 0.5255],
        [0.0000, 0.0000, 0.1430, 0.8266, 0.5940, 0.7007, 0.2518, 0.1667],
        [0.7255, 0.0080, 0.9489, 0.1894, 0.5512, 0.5293, 0.2753, 0.0508],
        [0.0584, 0.3933, 0.1139, 0.7904, 0.3747, 0.3514, 0.7094, 0.1210],
        [0.2017, 0.0492, 0.1117, 0.2658, 0.8969, 0.0542, 0.2911, 0.9022],
        [0.3773, 0.9827, 0.1489, 0.0572, 0.9670, 0.2209, 0.4054, 0.3755],
        [0.8901, 0.5765, 0.0042, 0.6808, 0.5876, 0.0787, 0.9588, 0.2276],
        [0.7936, 0.0282, 0.0572, 0.2471, 0.8485, 0.9265, 0.1902, 0.8694]])

In [2]:
from transformers import WhisperForConditionalGeneration

In [3]:
model = WhisperForConditionalGeneration.from_pretrained('/home/ujan/speech-processing/models/whisper/whisper-small-common_voice_13_0-sigmoied_threshold-pruned/pasted_checkpoint-5000')

In [46]:
model_old = WhisperForConditionalGeneration.from_pretrained('openai/whisper-small')

In [15]:
model.model.encoder.layers[7].self_attn.k_proj.weight

Parameter containing:
tensor([[-0.0000,  0.0000,  0.0000,  ..., -0.0388, -0.0000,  0.0000],
        [ 0.0000, -0.0000, -0.0000,  ..., -0.0000,  0.0231, -0.0000],
        [-0.0249, -0.0000,  0.0000,  ..., -0.0173, -0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000, -0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000]],
       requires_grad=True)

In [21]:
from transformers import AutoConfig, AutoFeatureExtractor, AutoTokenizer, AutoProcessor, AutoModelForSpeechSeq2Seq
from modeling_whisper_traceable import WhisperForConditionalGeneration
from datasets import load_dataset
import torch

model_name_or_path = 'openai/whisper-small'
data_dir = 'mozilla-foundation/common_voice_11_0'

In [48]:
# load dataset
print('loading dataset from {}'.format(data_dir))

raw_datasets = load_dataset(data_dir, "zh-CN", split="test", streaming=True)
text_column_name = 'sentence'


# model, tokenizer, feature extractor, processor


feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_name_or_path,
    #cache_dir=args.cache_dir,
    #revision=args.model_revision,
    #use_auth_token=True if args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    #cache_dir=args.cache_dir,
    #use_fast=model_args.use_fast_tokenizer,
    #revision=model_args.model_revision,
    #use_auth_token=True if model_args.use_auth_token else None,
)

tokenizer.set_prefix_tokens(language='chinese', task='transcribe')


    
processor = AutoProcessor.from_pretrained(model_name_or_path)


dataset = raw_datasets

model.eval()


sample = next(iter(dataset))


inputs = processor(
    sample['audio']["array"],
    sampling_rate=feature_extractor.sampling_rate,
    return_attention_mask=True,
    return_tensors="pt")
                
input_features = inputs.input_features
attention_mask = inputs.attention_mask
decoder_input_ids = torch.tensor([model.config.decoder_start_token_id]).reshape(1, -1)

loading dataset from mozilla-foundation/common_voice_11_0


Reading metadata...: 10581it [00:00, 23742.22it/s]


In [ ]:
%timeit output_ids = model.generate(input_features)

In [49]:
%timeit output_ids = model_old.generate(input_features)

546 ms ± 85.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
sample = next(iter(dataset))


inputs = processor(
    sample['audio']["array"],
    sampling_rate=feature_extractor.sampling_rate,
    return_attention_mask=True,
    return_tensors="pt")
                
input_features = inputs.input_features
attention_mask = inputs.attention_mask
decoder_input_ids = torch.tensor([model.config.decoder_start_token_id]).reshape(1, -1)

Reading metadata...: 10581it [00:00, 11039.05it/s]


In [36]:
%timeit output_ids = model.generate(input_features)

824 ms ± 8.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%timeit output_ids = model_old.generate(input_features)

791 ms ± 27.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
import transformers
import datasets
from transformers import WhisperForConditionalGeneration

In [3]:
model = WhisperForConditionalGeneration.from_pretrained('/home/ujan/speech-processing/models/whisper/whisper-small-common_voice_13_0-sigmoied_threshold-pruned/pasted_checkpoint-5000/')

In [9]:
model.model.encoder.layers[1].self_attn.k_proj.weight.shape

torch.Size([768, 768])

In [ ]:
from torch import nn
import torch

def prune_linear_layer(layer: nn.Linear, index: torch.LongTensor, dim: int = 0) -> nn.Linear:
    """
    Prune a linear layer to keep only entries in index.

    Used to remove heads.

    Args:
        layer (`torch.nn.Linear`): The layer to prune.
        index (`torch.LongTensor`): The indices to keep in the layer.
        dim (`int`, *optional*, defaults to 0): The dimension on which to keep the indices.

    Returns:
        `torch.nn.Linear`: The pruned layer as a new layer with `requires_grad=True`.
    """
    index = index.to(layer.weight.device)
    W = layer.weight.index_select(dim, index).clone().detach()
    if layer.bias is not None:
        if dim == 1:
            b = layer.bias.clone().detach()
        else:
            b = layer.bias[index].clone().detach()
    new_size = list(layer.weight.size())
    new_size[dim] = len(index)
    new_layer = nn.Linear(new_size[1], new_size[0], bias=layer.bias is not None).to(layer.weight.device)
    new_layer.weight.requires_grad = False
    new_layer.weight.copy_(W.contiguous())
    new_layer.weight.requires_grad = True
    if layer.bias is not None:
        new_layer.bias.requires_grad = False
        new_layer.bias.copy_(b.contiguous())
        new_layer.bias.requires_grad = True
    return new_layer

In [ ]:
m = nn.Linear(8, 16)

In [ ]:
index = torch.LongTensor([1,5,7])

In [ ]:
new_linear = prune_linear_layer(m, index)

In [ ]:
new_linear.weight.shape

In [ ]:
m.weight

In [ ]:
new_linear.weight

In [ ]:
n_heads = 12
heads = [1,3,7]
head_size = 64
already_pruned_heads = set()

mask = torch.ones(n_heads, head_size)
heads = set(heads) - already_pruned_heads  # Convert to set and remove already pruned heads
for head in heads:
    # Compute how many pruned heads are before the head and move the index accordingly
    head = head - sum(1 if h < head else 0 for h in already_pruned_heads)
    mask[head] = 0
mask = mask.view(-1).contiguous().eq(1)
index: torch.LongTensor = torch.arange(len(mask))[mask].long()

In [ ]:
len(index)

In [ ]:
index